The goal is to write a simple object detection pipeline invoking YOLOX.

In [1]:
%load_ext autoreload


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import torch
import torchvision
import os
import cv2
import motmetrics as mm
import importlib
from loguru import logger
import traceback

There are many YOLOX versions based on size, depth and efficiency and we select which yolox version to use by using the `get_exp` function provided by YOLOX.

In [4]:
import sys
sys.path.insert(0, "./YOLOX/")
from YOLOX.yolox.exp.build import get_exp_by_name
import src.detector as det
import src.tracker as track

In [5]:
YOLOX_VERSION = "yolox-s" #alternatives: yolox-m, yolox-l, yolox-x, yolox-tiny, yolox-nano, yolov3
yolo_exp = get_exp_by_name(YOLOX_VERSION)
detector = det.Detector(yolo_exp, filter_classes=None, device=torch.device("cuda"),
                 test_conf=0.25, nms_thres=0.65, class_agnostic=True, 
                 chkpt="yolo_weights/yolox_s.pth", num_classes=80)

In [6]:
img = cv2.imread("img/car_traffic.jpeg")

In [7]:
output, img_info = detector(img)

In [8]:
from src.visualizer import Visualizer

In [9]:
vis = Visualizer()
out_img = vis.add_boxes_to_img(output, img_info)

In [10]:
#cv2.imwrite("traffic_out.jpg", out_img)

In [11]:
my_tracker = track.Tracker(det=detector)

In [12]:


# cap = cv2.VideoCapture("img/example.mp4")
# width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) 
# height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) 
# fps = cap.get(cv2.CAP_PROP_FPS)
# time_elapsed = 0
# frame_count = 0
# cv2.namedWindow("yolox", cv2.WINDOW_NORMAL)
# while True:
#     ret_val, frame = cap.read()
#     frame_count += 1
#     if ret_val and (15 <= frame_count <= 17):
#         logger.info(f"Here!")
        
#         outputs, img_info = detector(frame)
#         result_frame = vis.add_boxes_to_img(outputs, img_info)
#         cv2.imshow("yolox", result_frame)
#         #if 15 <= frame_count <= 17:
#         #cv2.imshow("yolox", frame)
        
#         #if frame_count == 1:
#         #    cv2.waitKey(0)
#         cv2.waitKey(0)
        
#         # ch = cv2.waitKey(1)
#         # if ch == 27 or ch == ord("q") or ch == ord("Q"):
#         #     break
    
#     else:
#         continue
    
#     if frame_count >= 17:
#         break
    
# cap.release()
# cv2.destroyAllWindows()

In [13]:
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
cap = cv2.VideoCapture("img/MOT20-02-raw.webm")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH )) 
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
time_elapsed = 0
frame_count = 0
cv2.namedWindow("yolox", cv2.WINDOW_NORMAL)
detector = det.Detector(yolo_exp, filter_classes=[0], device=torch.device("cuda"),
                 test_conf=0.25, nms_thres=0.65, class_agnostic=True, 
                 chkpt="yolo_weights/yolox_s.pth", num_classes=80)
my_tracker = track.Tracker(det=detector)


LOWER_FRAME=16
HIGHER_FRAME=20
try:
    out = cv2.VideoWriter('img/MOT_test.avi',cv2.VideoWriter_fourcc(*'MJPG'), 60, (width, height))
    
    while cap.isOpened():
        ret_val, frame = cap.read()
        frame_count += 1
        if ret_val:
            #if (LOWER_FRAME <= frame_count <= HIGHER_FRAME):
                result_frame, curr_time = my_tracker.step(frame)
                time_elapsed += curr_time
                out.write(result_frame)
                cv2.imshow("yolox", result_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            #if frame_count >= HIGHER_FRAME: break
        
        else:
            break 
except Exception as e:
    logger.error(traceback.format_exc())
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
test_dir = "data/MOT20/MOT20/train/"
for vid_dir in os.listdir(test_dir):
    print(vid_dir)
    df_rows = []
    for i, file in enumerate(os.listdir(os.path.join(test_dir, vid_dir, "img1"))):
        frame = cv2.imread(os.path.join(test_dir, vid_dir, "img1", file))
        outputs, img_info = detector(frame)
        result_frame = vis.add_boxes_to_img(outputs[0], img_info)
        #cv2.namedWindow("yolox", cv2.WINDOW_NORMAL)
        #cv2.imshow("yolox", result_frame)
        #ch = cv2.waitKey(1)
        #if ch == 27 or ch == ord("q") or ch == ord("Q"):
        #    break
        outputs = outputs[0].cpu()
        outputs /= img_info["ratio"]
        for j, box in enumerate(outputs):
            row = [i, j] + box.numpy()[:4] + [1,-1,-1,-1]
            df_rows.append(row)
            break
#cv2.destroyAllWindows()

In [ ]:
for seqname in os.listdir(test_dir):
    pass